## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-13-08-51-18 +0000,nypost,Front-end loader helps elephant get back on he...,https://nypost.com/2025/09/13/us-news/milwauke...
1,2025-09-13-08-48-20 +0000,bbc,Starmer defended Mandelson after officials kne...,https://www.bbc.com/news/articles/c5yq455w36vo...
2,2025-09-13-08-42-17 +0000,bbc,Colombian court rules Meta was wrong to bar po...,https://www.bbc.com/news/articles/cp8wlxwy1exo...
3,2025-09-13-08-20-40 +0000,nyt,7.4-Magnitude Earthquake Strikes Russia’s Far ...,https://www.nytimes.com/2025/09/12/world/asia/...
4,2025-09-13-08-13-14 +0000,nyt,U.C. Berkeley Gives Names of Students and Facu...,https://www.nytimes.com/2025/09/13/us/politics...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2315/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
92,kirk,51
97,charlie,46
161,trump,23
205,shooting,14
206,suspect,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
287,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...,176
34,2025-09-13-02-26-24 +0000,wapo,Charlie Kirk shooting suspect in custody; fami...,https://www.washingtonpost.com/nation/2025/09/...,145
24,2025-09-13-03-51-29 +0000,nyt,"Tyler Robinson, Charlie Kirk Shooting Suspect,...",https://www.nytimes.com/2025/09/12/us/politics...,143
114,2025-09-12-20-56-19 +0000,bbc,"Who is Tyler Robinson, the suspect in custody ...",https://www.bbc.com/news/articles/cp8wl2y66p9o...,143
63,2025-09-13-00-01-58 +0000,nyt,Trump Blames the Left and Downplays ‘Radical R...,https://www.nytimes.com/2025/09/12/us/politics...,142


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
287,176,2025-09-12-09-30-00 +0000,wsj,"Charlie Kirk and JD Vance spoke regularly, tal...",https://www.wsj.com/politics/elections/jd-vanc...
37,57,2025-09-13-02-07-47 +0000,nypost,Missouri passes Trump-backed redistricting pla...,https://nypost.com/2025/09/12/us-news/missouri...
34,48,2025-09-13-02-26-24 +0000,wapo,Charlie Kirk shooting suspect in custody; fami...,https://www.washingtonpost.com/nation/2025/09/...
55,43,2025-09-13-00-40-07 +0000,nypost,Charlie Kirk’s wife Erika delivers emotional f...,https://nypost.com/2025/09/12/us-news/charlie-...
104,42,2025-09-12-21-08-55 +0000,nypost,Ukraine strikes key Russian oil terminal in la...,https://nypost.com/2025/09/12/world-news/ukrai...
211,39,2025-09-12-14-43-17 +0000,nypost,Warner Bros. Discovery CEO David Zaslav wants ...,https://nypost.com/2025/09/12/business/warner-...
199,36,2025-09-12-15-18-57 +0000,startribune,Another shocking act of political violence lea...,https://www.startribune.com/charlie-kirk-minne...
263,32,2025-09-12-10-56-14 +0000,nypost,Fashion designer Giorgio Armani’s will instruc...,https://nypost.com/2025/09/12/business/fashion...
106,31,2025-09-12-21-08-15 +0000,cbc,"Protests outside, silence inside as Canada goe...",https://www.cbc.ca/sports/tennis/canada-israel...
232,30,2025-09-12-12-50-11 +0000,nypost,New Joni Ernst bill would deactivate governmen...,https://nypost.com/2025/09/12/us-news/new-joni...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
